<font size="5">Ingest Time Series - GHG total without LULUCF, in kt CO₂ equivalent into Trino pipeline</font>


In [1]:
from dotenv import dotenv_values, load_dotenv
import osc_ingest_trino as osc
import os
import pathlib

Load Environment Variables

In [2]:
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [3]:
# use a catalog that is configured for iceberg
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'pcaf_sovereign_footprint'
ingest_table = 'sf_unfccc_without_lulucf'

In [4]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

trino_bucket = osc.attach_s3_bucket("S3_DEV")

In [5]:
import boto3

s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
source_bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

Open a Trino connection using JWT for authentication

In [6]:
# make sure schema exists, or table creation below will fail in weird ways
sql = f"""
create schema if not exists {ingest_catalog}.{ingest_schema}
"""
qres = engine.execute(sql)
print(qres.fetchall())

[(True,)]


In [7]:
# Show available schemas to ensure trino connection is set correctly
schema_read = engine.execute(f'show schemas in {ingest_catalog}')
for row in schema_read.fetchall():
    print(row)

('default',)
('demo_dv',)
('iceberg_demo',)
('information_schema',)
('ingest',)
('mdt_sandbox',)
('pcaf_sovereign_footprint',)
('sandbox',)


In [8]:
pip install country_converter --upgrade

Note: you may need to restart the kernel to use updated packages.


Load PCAF_UNFCC_WITHOUT_LULUCF

In [9]:
import pandas as pd
import ParseXLS as parser

ticker_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],'PCAF-sovereign-footprint/UNFCC/Time_Series_GHG_total_without_LULUCF_in_kt_CO₂_equivalent.xlsx')
ticker_file.download_file(f'/tmp/Time_Series_GHG_total_without_LULUCF_in_kt_CO₂_equivalent.xlsx')

df = parser.process('UNFCCC_without_LULUCF.ini','UNFCCC_without_LULUCF.csv') 

ticker_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],'PCAF-sovereign-footprint/UNFCC/Annual_Net_emissions_removals_in_Gg_CO₂_equivalent_excl_LULUCF.csv')
ticker_file.download_file(f'/tmp/Annual_Net_emissions_removals_in_Gg_CO₂_equivalent_excl_LULUCF.csv')

df2 = parser.process('UNFCCC_without_LULUCF_other.ini','UNFCCC_without_LULUCF_other.csv') 

# combine both dataframes

df =pd.concat([df,df2])

#pd.to_numeric(df["validity_date"],errors='raise')
df=df.astype({'validity_date': 'int32'})
df=df.astype({'country_iso_code': 'string'})
df=df.astype({'value': 'float'})

df= df.convert_dtypes()
df.info(verbose=True)
df= df[['rec_source','data_provider','country_iso_code','country_name','validity_date','attribute','value','value_units']].dropna(subset=['value'])






European Union (Convention) not found in regex
European Union (KP) not found in regex


UNFCCC_without_LULUCF.ini
file_list:
['/tmp/Time_Series_GHG_total_without_LULUCF_in_kt_CO₂_equivalent.xlsx']
/tmp/Time_Series_GHG_total_without_LULUCF_in_kt_CO₂_equivalent.xlsx
2
xls
/tmp/Time_Series_GHG_total_without_LULUCF_in_kt_CO₂_equivalen
['0', '2']
eval_components
['Time Series - GHG total without LULUCF, in kt CO₂ equivalent']
['Party', 'Base year', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', 'Last Inventory Year (2019)', 'Change from base year to latest reported year']
Time Series - GHG total without LULUCF, in kt CO₂ equivalent
eval_components
['kt CO2e']
['Party', 'Base year', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '

nan not found in ISO3
Source: UNFCCC GHG Data Interface not found in regex
Note 1: The reporting and review requirements for GHG inventories are different for Annex I and non-Annex I Parties. The definition format of data for emissions/removals from the forestry sector is different for Annex I and non-Annex I Parties. not found in regex
Note 2: Base year data in the data interface relate to the base year under the Climate Change Convention (UNFCCC).  The base year under the Convention is defined slightly different than the base year under the Kyoto Protocol.  An exception is made for European Union (KP) whereby the base year under the Kyoto Protocol is displayed. not found in regex
Note 3: – means "No data available" not found in regex
Note 4: Data displayed on the data interface are "as received" from Parties. The publication of Party submissions on this website does not imply the expression of any opinion whatsoever on the part of the UNFCCC or the Secretariat of the United Nations c

52
before melt
                                         country_name          1990  \
0                                           Australia  4.236722e+05   
1                                             Austria  7.842049e+04   
2                                             Belarus  1.391520e+05   
3                                             Belgium  1.457195e+05   
4                                            Bulgaria  9.997808e+04   
5                                              Canada  6.015237e+05   
6                                             Croatia  3.138729e+04   
7                                              Cyprus  5.571026e+03   
8                                             Czechia  1.970718e+05   
9                                             Denmark  7.109766e+04   
10                                            Estonia  4.104549e+04   
11                        European Union (Convention)  5.657987e+06   
12                                European Union (KP)  5.66439

Party \ Unit not found in regex
Note 1: The reporting and review requirements for GHG inventories are different for Annex I and non-Annex I Parties. The definition format of data for emissions/removals from the forestry sector is different for Annex I and non-Annex I Parties. not found in regex
Note 2: Base year data in the data interface relate to the base year under the Climate Change Convention (UNFCCC).  The base year under the Convention is defined slightly different than the base year under the Kyoto Protocol.  An exception is made for European Union (KP) whereby the base year under the Kyoto Protocol is displayed. not found in regex
More then one regular expression match for Note 3: Some non-Annex I Parties submitted their GHG inventory data using  the format of the 2006 IPCC Guidelines in reporting GHG emissions/removals.  For this reason, these data could not be included in the data interface.  However, the data are available in the national communications (Andorra, Angola,  A

156


More then one regular expression match for Note 3: Some non-Annex I Parties submitted their GHG inventory data using  the format of the 2006 IPCC Guidelines in reporting GHG emissions/removals.  For this reason, these data could not be included in the data interface.  However, the data are available in the national communications (Andorra, Angola,  Antigua and Barbuda, Armenia, Azerbaijan, Bahrain, Bangladesh, Bhutan, Brazil, Brunei Darussalam, Cabo Verde, Cook Islands, Costa Rica,  Côte d'Ivoire, Colombia, Cuba, Equatorial Guinea, Eswatini, Fiji, Gabon, Gambia, Georgia, Grenada, Ghana, Honduras, Indonesia, Iran, Jamaica, Kuwait, Lesotho,  Malaysia, Mauritania, Mauritius, Mexico, Mongolia, Montenegro, Morocco, Namibia, Nepal, Nicaragua, Nigeria, Panama, Oman,  Republic of Moldova, Rwanda, Samoa, Serbia, Sierra Leone, Singapore, Somalia,  South Africa, Suriname, Timor-Leste, Trinidad and Tobago, United Arab Emirates, Vanuatu, Venezuela, Viet Nam, and Zambia)  and biennial update reports

156
before melt
                                          country_name               1990  \
0                                         Party \ Unit  Gg CO₂ equivalent   
1                                          Afghanistan                NaN   
2                                              Albania            4341.63   
3                                              Algeria                NaN   
4                                               Angola                NaN   
..                                                 ...                ...   
151  Note 3: Some non-Annex I Parties submitted the...               None   
152                Note 4: – means "No data available"               None   
153  Note 5: Data displayed on the data interface a...               None   
154                  Source: UNFCCC GHG Data Interface               None   
155  Report produced on Friday, 10 June 2022, 14:44...               None   

                  1991               1992               199

In [10]:
import osc_ingest_trino as osc
columnschema = osc.create_table_schema_pairs(df) 

sql = f"""
drop table if exists {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
print(sql)
qres = engine.execute(sql)
print(qres.fetchall())



drop table if exists osc_datacommons_dev.pcaf_sovereign_footprint.sf_unfccc_without_lulucf

[(True,)]


In [11]:
tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
    format = 'ORC',
    partitioning = array['country_iso_code']
)
"""
print(tabledef)
qres = engine.execute(tabledef)
print(qres.fetchall())


create table if not exists osc_datacommons_dev.pcaf_sovereign_footprint.sf_unfccc_without_lulucf(
    rec_source varchar,
    data_provider varchar,
    country_iso_code varchar,
    country_name varchar,
    validity_date integer,
    attribute varchar,
    value double,
    value_units varchar
) with (
    format = 'ORC',
    partitioning = array['country_iso_code']
)

[(True,)]


In [12]:
# Delete all data from our db, so we start with empty table
sql=f"""
delete from {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
qres = engine.execute(sql)
print(qres.fetchall())

[(None,)]


In [13]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
pd.read_sql(sql, engine)


,rec_source,data_provider,country_iso_code,country_name,validity_date,attribute,value,value_units


In [14]:
print(ingest_catalog)
#df=df.drop(df[df.country_name=="cote d'ivoire"].index)
df.to_sql(ingest_table,
           con=engine,
           schema=ingest_schema,
           if_exists='append',
           index=False,
           method=osc.TrinoBatchInsert(batch_size = 1000, verbose = True))

osc_datacommons_dev
constructed fully qualified table name as: "pcaf_sovereign_footprint.sf_unfccc_without_lulucf"
inserting 1000 records
  ('Time_Series_GHG_total_without_LULUCF_in_kt_CO₂_equivalent.xlsx', 'UNFCCC', 'AUS', 'Australia', 1990, 'Time Series - GHG total without LULUCF, in kt CO₂ equivalent', 423672.2245458623, 'kt CO2e')
  ('Time_Series_GHG_total_without_LULUCF_in_kt_CO₂_equivalent.xlsx', 'UNFCCC', 'AUT', 'Austria', 1990, 'Time Series - GHG total without LULUCF, in kt CO₂ equivalent', 78420.48531347305, 'kt CO2e')
  ('Time_Series_GHG_total_without_LULUCF_in_kt_CO₂_equivalent.xlsx', 'UNFCCC', 'BLR', 'Belarus', 1990, 'Time Series - GHG total without LULUCF, in kt CO₂ equivalent', 139151.98563558093, 'kt CO2e')
  ...
  ('Time_Series_GHG_total_without_LULUCF_in_kt_CO₂_equivalent.xlsx', 'UNFCCC', 'EST', 'Estonia', 2013, 'Time Series - GHG total without LULUCF, in kt CO₂ equivalent', 22019.83909772197, 'kt CO2e')
batch insert result: [(1000,)]
inserting 1000 records
  ('Time_Se

In [15]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}"""
pd.read_sql(sql, engine)


,rec_source,data_provider,country_iso_code,country_name,validity_date,attribute,value,value_units
0,Time_Series_GHG_total_without_LULUCF_in_kt_CO₂...,UNFCCC,POL,Poland,1990,"Time Series - GHG total without LULUCF, in kt ...",4.758629e+05,kt CO2e
1,Time_Series_GHG_total_without_LULUCF_in_kt_CO₂...,UNFCCC,POL,Poland,1991,"Time Series - GHG total without LULUCF, in kt ...",4.637909e+05,kt CO2e
2,Time_Series_GHG_total_without_LULUCF_in_kt_CO₂...,UNFCCC,POL,Poland,1992,"Time Series - GHG total without LULUCF, in kt ...",4.510159e+05,kt CO2e
3,Time_Series_GHG_total_without_LULUCF_in_kt_CO₂...,UNFCCC,POL,Poland,1993,"Time Series - GHG total without LULUCF, in kt ...",4.509113e+05,kt CO2e
4,Time_Series_GHG_total_without_LULUCF_in_kt_CO₂...,UNFCCC,POL,Poland,1994,"Time Series - GHG total without LULUCF, in kt ...",4.452890e+05,kt CO2e
...,...,...,...,...,...,...,...,...
2207,Time_Series_GHG_total_without_LULUCF_in_kt_CO₂...,UNFCCC,JPN,Japan,2008,"Time Series - GHG total without LULUCF, in kt ...",1.319258e+06,kt CO2e
2208,Time_Series_GHG_total_without_LULUCF_in_kt_CO₂...,UNFCCC,JPN,Japan,2009,"Time Series - GHG total without LULUCF, in kt ...",1.247101e+06,kt CO2e
2209,Time_Series_GHG_total_without_LULUCF_in_kt_CO₂...,UNFCCC,JPN,Japan,2010,"Time Series - GHG total without LULUCF, in kt ...",1.300452e+06,kt CO2e
2210,Time_Series_GHG_total_without_LULUCF_in_kt_CO₂...,UNFCCC,JPN,Japan,2011,"Time Series - GHG total without LULUCF, in kt ...",1.351252e+06,kt CO2e
